### 01. 모듈 import / AP몰 접속 / 오늘 하루 다시보지 않기 / brand name 검색

In [268]:
########## 모듈 import ##########
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver as wd
import time

driver = wd.Chrome(executable_path='/Users/hyunginchoi/Desktop/com/python/chromedriver')



########## url과 brand 설정 ##########
ap_url = 'https://www.amorepacificmall.com'
brand_name = '이니스프리'



########## url 접속 후 brand name 검색 ##########
driver.get(ap_url)

driver.find_element_by_class_name('layer_close').click()

driver.find_element_by_id('query_str').send_keys(brand_name)
driver.find_element_by_class_name('btn_search').click()

### 02. 제품 ID 리스트에 저장 / 페이지 이동 반복 / 페이지가 끝나면 except

In [231]:
obj_id = []
try:
    while True:
        ########## 각 제품의 id get ##########
        prd_cell = driver.find_elements_by_class_name('item.item-apply')
        for prd in prd_cell:
            obj_id.append(prd.find_element_by_css_selector('a').get_attribute('href')[-5:])
        
        
        ########## 다음 페이지로 이동 ##########
        driver.find_element_by_css_selector('span + a').click()
        time.sleep(0.3)

########## 다음 페이지가 없을 때 발생하는 에러 ##########
except NoSuchElementException as e:
    print('마지막 페이지입니다')

마지막 페이지입니다


### 03. 데이터 저장할 text file 생성

In [270]:
prd_info = open("apmall_prd_info.txt", "a")
prd_reviews = open("apmall_prd_reviews.txt", "a")

### 04. 각 제품 url에 접속 / 정보 크롤링 / write

In [ ]:
k = 1
for i in obj_id:
    ########## 얻은 제품 id로 각 제품 url에 접속 ##########
    driver.get(ap_url + '/kr/ko/product/detail?onlineProdSn=' + i)
    time.sleep(1)
    
    ########## name, price, review count, point ##########
    prd_name = driver.find_element_by_class_name('product_name').text
    prd_price = driver.find_element_by_class_name('product_price').find_element_by_class_name('num').text
    review_count = driver.find_element_by_class_name('review.visibleReview').find_element_by_class_name('txt').text[3:]
    point = driver.find_element_by_css_selector('div > div.detail_head > div > div.product_info > div.purchase_benefit > p').text
    
    shipping_btn = driver.find_element_by_css_selector('#ap_container > div > div.detail_head > div > div.product_info > div.purchase_benefit > div > dl:nth-child(1) > dt > button').click()
    shipping = driver.find_element_by_css_selector('#ap_container > div > div.detail_head > div > div.product_info > div.purchase_benefit > div > dl:nth-child(1) > dd > div > p').text
    
    purchase_benefit_btn = driver.find_element_by_css_selector('#ap_container > div > div.detail_head > div > div.product_info > div.purchase_benefit > div > dl:nth-child(2) > dt > button').click()
    purchase_benefits = driver.find_elements_by_css_selector('#ap_container > div > div.detail_head > div > div.product_info > div.purchase_benefit > div > dl:nth-child(2) > dd > div > ul > li')
    benefit_total = ''
    for benefit in purchase_benefits:
        span = benefit.find_element_by_css_selector('span').text
        em = '(' + benefit.find_element_by_css_selector('em').text + ')'
        benefit_total += span + em + " "
    

    
    ########## 모든 리뷰가 보일 때까지 '더보기' 클릭 ##########
    review_click = driver.find_element_by_css_selector('#ap_container > div > div.detail_body.ui_tab.＠tabs-apply > div.tab_menu > ul > li:nth-child(2) > button').click()
    try:
        while True:
            driver.find_element_by_class_name('btn_list_more').click()
            time.sleep(1)
    except:
        print("-")
    
    
    ########## date, rating, user_id, text ##########
    review_box = driver.find_elements_by_class_name('review_box')
    j = 1
    total_rating = 0
    for review in review_box:
        date = review.find_element_by_class_name('rating').find_element_by_class_name('date').text
        rating = review.find_element_by_class_name('rating').find_element_by_css_selector('span:nth-child(1)').get_attribute('class')[-1]
        user_id = review.find_element_by_class_name('name').text
        text = review.find_element_by_class_name('ellipsis.line5').text
        total_rating += int(rating)
        prd_reviews.write(str(k) + " | " + str(j) + " | " + date + " | " + rating + " | " + user_id + " | " + text + "\n")
        j += 1
    prd_reviews.write("\n")    
    
    if review_count == '0':
        rating_avg = '0.00'
    else:
        rating_avg = str("%.2f" % (total_rating / int(review_count)))
    
    prd_info.write(str(k) + " | " + prd_name + " | " + prd_price + " | " + rating_avg + " | " + point[:9] + " | " + shipping_cond[9:28] + " | " + benefit_total + "\n")

    k += 1
    
prd_info.close()
prd_reviews.close()
    

In [151]:
driver.close()